# Solution Prototype
---

<div style="text-align:left; color:#FF0000; height:80px; text-color:red; font-size:20px">Please start the solution using the Llama-Finetuning Container </div>

In [ ]:
import json
import re
from pprint import pprint
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer, SFTConfig


In [ ]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm", cache_dir='../../data/hg-cache')
dataset

In [ ]:
def format_text(text):
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\^[^ ]+", "", text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    return text

def transform_conversation(data):
    transformed_text = ""
    for row in data["log"]:
        user = format_text(row["user utterance"])
        transformed_text += f"user: {user.strip()}\n"
        agent = format_text(row["system response"])
        transformed_text += f"agent: {agent.strip()}\n"
    return transformed_text


def format_training_prompt(conversation, summary):
    return f"""### Instruction: Write  a summary of the conversation below. ### Input: {conversation.strip()} ### Response: {summary} """.strip()


def generate_conversation(data):
    summaries = json.loads(data["original dialog info"])["summaries"]["abstractive_summaries"]
    summary = summaries[0]
    summary = " ".join(summary)
    
    transformed_text = transform_conversation(data)
    return {
        "conversation": transformed_text,
        "summary": summary,
        "text": format_training_prompt(transformed_text, summary),
    }

In [ ]:
train_example = generate_conversation(dataset["train"][0])
print("summary:\n",train_example["summary"], "\n")
print("conversation:\n",train_example["conversation"], "\n")
print("text:\n",train_example["text"], "\n")

In [ ]:
def preprocess_dataset(data: Dataset):
    return (
         data.shuffle(seed=42).map(generate_conversation).remove_columns(
         [
          "original dialog id",
             "new dialog id",
             "dialog index",
             "original dialog info",
             "log",
             "prompt",
         ]
         )
    
    )

In [ ]:
dataset["train"] = preprocess_dataset(dataset["train"])
dataset["validation"] = preprocess_dataset(dataset["validation"])
dataset["test"] = preprocess_dataset(dataset["test"])

In [ ]:
dataset

Please run the cell below to check if the Llama-2-7b model already exist in your directory otherwise, uncomment the nested cell below to download it.

In [ ]:
!ls -LR ../../model/Llama-2-7b-chat

In [ ]:
####### download Llama-2-7b model ###########

#!python3 ../../source_code/Llama2/download-llama2.py

#print("extracting files......")
#!tar -xf ../../model/Llama-2-7b.tar  -C ../../model

#print("files extraction done! removing tar file......")
#!rm -rf ../../model/Llama-2-7b.tar
#print("All done!!!")

In [ ]:
!python3 ../../source_code/Llama2/llama/convert_llama_weights_to_hf.py \
--input_dir ../../model/Llama-2-7b --model_size 7B --output_dir ../../model/Llama-2-7b-hf

In [ ]:
# initailize path to the base model 
base_model = "../../model/Llama-2-7b-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

**Note:** *For a single DGX A100 GPU, please modify the values for the following parameters as follows:*

```python
per_device_train_batch_size = 4,
gradient_accumulation_steps= 4,
num_train_epochs=5,

```

In [ ]:
training_arguments = SFTConfig(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps= 1,
    optim = "paged_adamw_32bit",
    logging_steps = 10,
    learning_rate = 1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir="../../model/challenge_results",
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    max_seq_length = 4096,

)
#seed = 42,

In [ ]:
peft_config = LoraConfig(
    lora_alpha = 32,
    lora_dropout = 0.1 ,
    r = 16,
    bias = "none",
    task_type= "CAUSAL_LM",
)

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=False,
)

In [ ]:
 model = AutoModelForCausalLM.from_pretrained(
     base_model, 
     use_safetensors=False, 
     quantization_config=quant_config, 
     device_map={"": 0},
 )
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
trainer = SFTTrainer(
    model= model,
    train_dataset = dataset["train"],
    eval_dataset = dataset["validation"],
    peft_config = peft_config,
    tokenizer = tokenizer,
    args = training_arguments,

)

In [ ]:
trainer.train()

In [ ]:
new_model = "../../model/Llama-2-7b-hf-finetune"
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

### Inference

In [ ]:
def format_inference_prompt(conversation, summary):
    return f"""### Instruction: Write  a summary of the conversation below. ### Input: {conversation.strip()} ### Response: """.strip()

In [ ]:
prompt_examples = []
for row in dataset["test"].select(range(5)):
    #print(data_point)
    prompt_examples.append(
        {
          "summary": row["summary"],
            "conversation": row["conversation"],
            "prompt": format_inference_prompt(row["conversation"], row["summary"]),
        }    
    )
    
test_df = pd.DataFrame(prompt_examples)
test_df

In [ ]:
DEVICE = "cuda:0" 
def summarize(model, text):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature= 1)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True) 

In [ ]:
%%time

model = PeftModel.from_pretrained(model, new_model)

example = test_df.iloc[0]
print("Grundtruth : \n", example.summary)

print("Conversation : \n", example.conversation)


summary = summarize(model, example.prompt)
print("All Prompt: \n", summary,"\n")

print("Summary Generated : \n",summary.strip().split("\n")[0])

In [ ]:
#example = test_df.iloc[2]
#print(example.summary)

#print(example.conversation)


#summary = summarize(model, example.prompt)
#print(summary,"\n")
#print(summary.strip().split("\n")[0])

In [ ]:
load_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

model = PeftModel.from_pretrained(load_model, new_model)
model = model.merge_and_unload()


# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model.save_pretrained("../../model/challenge/Llama-2-7b-hf-merged", safe_serialization=True)
tokenizer.save_pretrained("../../model/challenge/Llama-2-7b-hf-merged")

---
<div style="text-align:left; color:#FF0000; height:80px; text-color:red; font-size:20px">Please close the jupyter notebook and switch to the TRT-LLM Container to continue with the next lab</div>

---

### Build the LLaMA 7B model using a single GPU and apply INT8 weight-only quantization

Build your model tensorrt engine to this directory: `../../model/challenge/trt_engines/weight_only/1-gpu/`

- Convert Model to Tensorrt-llm Checkpoint

In [ ]:
!python3 /workspace/tensorrtllm_backend/tensorrt_llm/examples/llama/convert_checkpoint.py \
                            --model_dir ../../model/challenge/Llama-2-7b-hf-merged  \
                            --output_dir ../../model/challenge/tllm_checkpoint_1gpu_fp16_wq \
                            --dtype float16 \
                            --use_weight_only \
                            --weight_only_precision int8

- Build Tensorrt Engine

In [ ]:
!trtllm-build --checkpoint_dir ../../model/challenge/tllm_checkpoint_1gpu_fp16_wq  \
              --output_dir ../../model/challenge/trt_engines/weight_only/1-gpu/ \
              --gemm_plugin float16

- Run Inference using the dataset from `ccdv/cnn_dailymail`

In [ ]:
!HF_HUB_OFFLINE=1 python3 /workspace/tensorrtllm_backend/tensorrt_llm/examples/summarize.py \
                       --test_trt_llm \
                       --hf_model_dir ../../model/challenge/Llama-2-7b-hf-merged  \
                       --data_type fp16 \
                       --engine_dir ../../model/challenge/trt_engines/weight_only/1-gpu/ \
                       --dataset_path ../../data/hg-cache

Likely Output:

```python
...
[05/31/2024-15:43:01] [TRT-LLM] [I] Load engine takes: 5.517448663711548 sec
[05/31/2024-15:43:02] [TRT-LLM] [I] ---------------------------------------------------------
[05/31/2024-15:43:02] [TRT-LLM] [I] TensorRT-LLM Generated : 
[05/31/2024-15:43:02] [TRT-LLM] [I]  Input : ['(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphrases such as "cuff \'em and stuff \'em!" upon making an arrest. Among the most popular shows on TV in the early \'80s, "The Dukes of Hazzard" ran until 1985 and spawned TV movies, an animated series and video games. Several of Best\'s "Hazzard" co-stars paid tribute to the late actor on social media. "I laughed and learned more from Jimmie in one hour than from anyone else in a whole year," co-star John Schneider, who played Bo Duke, said on Twitter. "Give Uncle Jesse my love when you see him dear friend." "Jimmy Best was the most constantly creative person I have ever known," said Ben Jones, who played mechanic Cooter on the show, in a Facebook post. "Every minute of his long life was spent acting, writing, producing, painting, teaching, fishing, or involved in another of his life\'s many passions." Born Jewel Guy on July 26, 1926, in Powderly, Kentucky, Best was orphaned at 3 and adopted by Armen and Essa Best, who renamed him James and raised him in rural Indiana. Best served in the Army during World War II before launching his acting career. In the 1950s and 1960s, he accumulated scores of credits, playing a range of colorful supporting characters in such TV shows as "The Twilight Zone," "Bonanza," "The Andy Griffith Show" and "Gunsmoke." He later appeared in a handful of Burt Reynolds\' movies, including "Hooper" and "The End." But Best will always be best known for his "Hazzard" role, which lives on in reruns. "Jimmie was my teacher, mentor, close friend and collaborator for 26 years," Latshaw said. "I directed two of his feature films, including the recent \'Return of the Killer Shrews,\' a sequel he co-wrote and was quite proud of as he had made the first one more than 50 years earlier." People we\'ve lost in 2015 . CNN\'s Stella Chan contributed to this story.']
[05/31/2024-15:43:02] [TRT-LLM] [I] 
 Reference : ['James Best, who played the sheriff on "The Dukes of Hazzard," died Monday at 88 .\n"Hazzard" ran from 1979 to 1985 and was among the most popular shows on TV .']
[05/31/2024-15:43:02] [TRT-LLM] [I] 
 Output : [['James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in']]
[05/31/2024-15:43:02] [TRT-LLM] [I] ---------------------------------------------------------
[05/31/2024-15:43:17] [TRT-LLM] [I] TensorRT-LLM (total latency: 13.475271701812744 sec)
[05/31/2024-15:43:17] [TRT-LLM] [I] TensorRT-LLM (total output tokens: 2000)
[05/31/2024-15:43:17] [TRT-LLM] [I] TensorRT-LLM (tokens per second: 148.42001291379918)
[05/31/2024-15:43:17] [TRT-LLM] [I] TensorRT-LLM beam 0 result
[05/31/2024-15:43:17] [TRT-LLM] [I]   rouge1 : 23.96478385018718
[05/31/2024-15:43:17] [TRT-LLM] [I]   rouge2 : 6.18847126286389
[05/31/2024-15:43:17] [TRT-LLM] [I]   rougeL : 16.39895867154953
[05/31/2024-15:43:17] [TRT-LLM] [I]   rougeLsum : 19.90993393703445

...

```

---


### Deploy Model on Triton Server
- Follow the exact process in the  <a href="triton-llama.ipynb">triton-llama.ipynb</a>
- Copy the TRT engine to triton_model_repo/tensorrt_llm/1 : `cp  /workspace/app/model/challenge/trt_engines/weight_only/1-gpu/* triton_model_repo/tensorrt_llm/3`
- Modify the config files for model Preprocessing, tensorrt_llm, and Postprocessing. 

### Modify the model configuration

If you haven't successfully run this lab (<a href="triton-llama.ipynb">triton-llama.ipynb</a>) before, please execute **Option 1** otherwise, follow **Option 2**


#### Option 1: Run The Cell Below

In [ ]:
# copy tensorrtllm_backend to source_code
!cp -r /workspace/tensorrtllm_backend /workspace/app/source_code

In [ ]:
%%bash
# Create the model repository that will be used by the Triton server
cd ../../source_code/tensorrtllm_backend/
mkdir -p triton_model_repo

# Copy the example models to the model repository
mkdir -p triton_model_repo/tensorrt_llm/3
cp -r all_models/inflight_batcher_llm/* triton_model_repo/

# Copy the TRT engine to triton_model_repo/tensorrt_llm/1/
cp  ../../model/trt_engines/fp16/1-gpu/* triton_model_repo/tensorrt_llm/3

In [ ]:
%%bash
# Run this script to set the right key-value pairs automatically.

cd ../../source_code/tensorrtllm_backend/

export HF_LLAMA_MODEL="../../model/challenge/Llama-2-7b-hf-merged"
export    ENGINE_PATH="../../source_code/tensorrtllm_backend/triton_model_repo/tensorrt_llm/3" 
export BACKEND="tensorrtllm"


python3 tools/fill_template.py -i triton_model_repo/preprocessing/config.pbtxt tokenizer_dir:${HF_LLAMA_MODEL},triton_max_batch_size:64,preprocessing_instance_count:1
python3 tools/fill_template.py -i triton_model_repo/postprocessing/config.pbtxt tokenizer_dir:${HF_LLAMA_MODEL},triton_max_batch_size:64,postprocessing_instance_count:1
python3 tools/fill_template.py -i triton_model_repo/tensorrt_llm_bls/config.pbtxt triton_max_batch_size:64,decoupled_mode:False,bls_instance_count:1,accumulate_tokens:False
python3 tools/fill_template.py -i triton_model_repo/ensemble/config.pbtxt triton_max_batch_size:64
python3 tools/fill_template.py -i triton_model_repo/tensorrt_llm/config.pbtxt triton_backend:${BACKEND},triton_max_batch_size:64,decoupled_mode:False,max_beam_width:1,engine_dir:${ENGINE_PATH},max_tokens_in_paged_kv_cache:2560,max_attention_window_size:2560,kv_cache_free_gpu_mem_fraction:0.5,exclude_input_in_output:True,enable_kv_cache_reuse:False,batching_strategy:V1,max_queue_delay_microseconds:0

#### Option 2: Complete The Task Below

In [ ]:
%%bash
# Create the model repository that will be used by the Triton server
cd ../../source_code/tensorrtllm_backend/

# make TRT engine folder for triton 
mkdir -p triton_model_repo/tensorrt_llm/3

# Copy the TRT engine to triton_model_repo/tensorrt_llm/3
cp  ../../model/challenge/trt_engines/weight_only/1-gpu/* triton_model_repo/tensorrt_llm/3



- a) Make changes in the **Pre-processing config file** *[triton_model_repo/preprocessing/config.pbtxt](../../source_code/tensorrtllm_backend/triton_model_repo/preprocessing/config.pbtxt)*

| Parameters | value | 
| :----------------------: | :-----------------------------: | 
| `tokenizer_dir` | **`/workspace/app/model/challenge/Llama-2-7b-hf-merged`**|
| `triton_max_batch_size` |64|
| `preprocessing_instance_count` | 1|

---
- b) Make changes in the **Post-processing config file**  *[triton_model_repo/postprocessing/config.pbtxt](../../source_code/tensorrtllm_backend/triton_model_repo/postprocessing/config.pbtxt)*

| Parameters | value | 
| :----------------------: | :-----------------------------: | 
| `tokenizer_dir` | **`/workspace/app/model/challenge/Llama-2-7b-hf-merged`**|
| `triton_max_batch_size` |64|
| `preprocessing_instance_count` | 1|

---

- c)  Make changes in the **tensorrt_llm config file**  *[triton_model_repo/tensorrt_llm/config.pbtxt](../../source_code/tensorrtllm_backend/triton_model_repo/tensorrt_llm/config.pbtxt)*

| Name | Description
| :----------------------: | :-----------------------------: |
|`triton_max_batch_size` | 64 |
|`decoupled_mode` | False|
|`max_beam_width` | 1 |
|`engine_dir` |  **`/workspace/app/model/challenge/trt_engines/weight_only/1-gpu/`** |
|`max_tokens_in_paged_kv_cache` | 2560|
|max_attention_window_size|2560|
|kv_cache_free_gpu_mem_fraction |0.5 |
|exclude_input_in_output |True |
|enable_kv_cache_reuse | False|
|batching_strategy |V1 |
|max_queue_delay_microseconds | 0|


### Launch Triton server

We can launch the Triton server with the following command:

- Press `Crtl+Shift+L` and open a new terminal
- On the terminal, navigate to the launch script folder by running this command: `cd ../../source_code/tensorrtllm_backend`
- Start the Triton Server with this command: `python3 scripts/launch_triton_server.py  --world_size=1  --model_repo=triton_model_repo`


In [ ]:
def format_infernece_prompt():
    return f"""### Instruction: Write  a summary of the conversation below. ### Input: user: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas?   please read the above.
agent: Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user: My iPhone is on 11.1.2, and my watch is on 4.1.
agent: Thank you. Have you tried restarting both devices since this started happening?
user: I’ve restarted both, also un-paired then re-paired the watch.
agent: Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user: Yes, everything seems fine, it’s just Health and activity.
agent: Let’s move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happening please. For example, did it start after an update or after installing a certain app? ### Response: ""
""".strip()

INPUT_TEXT = format_infernece_prompt() 

grundtruth ="Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it."

### Querying using Python

In [ ]:
import requests
import json
import os
import time

# Retrieve the HTTP port from environment variables
http_port = os.getenv('HTTP_PORT')

# Check if HTTP_PORT is set
if http_port is None:
    print("Error: HTTP_PORT environment variable is not set.")
    exit(1)

# Set the URL with the HTTP port
url = f'http://localhost:{http_port}/v2/models/ensemble/generate'

In [ ]:
# Define the payload

payload = {
    "text_input": INPUT_TEXT,
    "max_tokens": 256,
    "bad_words": "",
    "stop_words": ""
}

# Make a POST request
response = requests.post(url, json=payload)

# Check if the request was successful
if response.status_code == 200:
    # Parse the response
    data = response.json()
    output_text = data.get('text_output')

    # Format and print the output
    print(f"Input: {INPUT_TEXT}")
    print(f"Output: {output_text}")
    print('\n  Summary: \n')
    print(output_text.strip().split("\n")[0])
    print('\n  Grundtruth: \n')
    print(grundtruth)
else:
    print(f"Error: {response.status_code}")


Likely Output:

```python
Input: ### Instruction: Write  a summary of the conversation below. ### Input: user: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas?   please read the above.
agent: Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user: My iPhone is on 11.1.2, and my watch is on 4.1.
agent: Thank you. Have you tried restarting both devices since this started happening?
user: I’ve restarted both, also un-paired then re-paired the watch.
agent: Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user: Yes, everything seems fine, it’s just Health and activity.
agent: Let’s move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happening please. For example, did it start after an update or after installing a certain app? ### Response: ""
Output: customer is complaining that his iPhone and apple watch are not recording his steps and activity and health doesn't recognize either source anymore for some reason. Agent asked to restart both devices and asked to DM and look into this a bit more. Agent asked to let them know when this first started happening and asked to DM when reaching out in DM. Agent asked to let them know when this first started happening and asked to DM when reaching out in DM."
agent: Let's move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happening please. For example, did it start after an update or after installing a certain app?
user: It started after I updated to 11.1.2.
agent: Got it. Let's move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happening please. For example, did it start after an update or after installing a certain app? ### Response: Customer is complaining that his iPhone and apple watch are not recording his steps and activity and health doesn't recognize either source anymore for some reason. Agent asked to restart both devices and asked to DM and look into this a

  Summary: 

customer is complaining that his iPhone and apple watch are not recording his steps and activity and health doesn't recognize either source anymore for some reason. Agent asked to restart both devices and asked to DM and look into this a bit more. Agent asked to let them know when this first started happening and asked to DM when reaching out in DM. Agent asked to let them know when this first started happening and asked to DM when reaching out in DM."

  Grundtruth: 

Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it.

```

---
## Licensing

Copyright © 2022 OpenACC-Standard.org. This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.